In [2]:
import nltk
# nltk.download('punkt')
import numpy as np
from nltk.tokenize import sent_tokenize
import json
import itertools
import re
import os
from collections import defaultdict

In [3]:
# base_path = "../../data/test/"
# file_name = "164726"
base_path = "../../../train/"

In [4]:
all_files = os.listdir(base_path)
filenames = set([name.split(".")[0] for name in all_files])

In [5]:
def parse_entities_relationships(file_content):
    """
    parses out entities and relationships from the entities file
    """
    entities = {}
    relationships = {}

    for line in file_content.strip().split('\n'):
        # madarchod line might be annotator comment
        if line.strip().startswith("#"):
            continue
        cols = line.split('\t')
        identifier = cols[0]
        if "Arg1:" in cols[1]:
            relationship_type, arg1, arg2 = cols[1].split()
            relationships[identifier] = {
                'type': relationship_type,
                'arg1': arg1.split(':')[1],
                'arg2': arg2.split(':')[1]
            }
            
        else:
            split_cols = cols[1].split()
            entity_type = split_cols[0]
            value = cols[2]
            start = split_cols[1]
            end = split_cols[-1]
            entities[identifier] = {
                'type': entity_type,
                'start': int(start),
                'end': int(end),
                'value': value
            }

    return entities, relationships

In [12]:
def get_spans(entities, relationships, txt_str, split_char="\n"):
    """
    first pass through spans
    """
    spans = {}
    for rel in relationships.keys():
        arg1 = entities[relationships[rel]['arg1']]
        arg2 = entities[relationships[rel]['arg2']]
        min_left = min(arg1['start'], arg2['start'])
        max_right = max(arg1['end'], arg2['end'])
        # find dots
        prev_char = txt_str[:min_left].rfind(split_char)
        next_char = txt_str[max_right:].find(split_char)+max_right
        # see if no exist
        assert(prev_char != -1)
        assert(next_char != -1)
        # see that span is not insanely large
        if max_right-min_left > 10:
            assert((next_char-prev_char) <= 10*(max_right-min_left))
        # return spans
        spans[rel] = {
            'start':prev_char,
            'end':next_char
        }
    return spans

In [13]:
def get_ents_in_spans(spans, ents):
    span_ent = defaultdict(list)
    for span in spans.keys():
        s = spans[span]
        for ent in ents.keys():
            e = ents[ent]
            if s['start'] <= e['start'] and s['end'] >= e['end']:
                span_ent[span].append(ent)
    return span_ent

In [14]:
# CHECKING FOR ALL FILES MA CHUDAO
for file_name in filenames:
    print(file_name)
    # read annotations
    ann_str = None
    with open(base_path + file_name + ".ann", "r") as file:
        ann_str = file.read()
    # parse entities and relations
    ent, rel = parse_entities_relationships(ann_str)
    # parse text string
    txt_str = None
    with open(base_path + file_name + ".txt", "r") as file:
        txt_str = file.read()
    # first pass through spans
    spans = get_spans(ent, rel, txt_str, "\n")
    # second pass through spans
    span_ents = get_ents_in_spans(spans, ent)
    print(spans)
    print(span_ents)
    print()

109566
{'R1': {'start': 9968, 'end': 10031}, 'R2': {'start': 9968, 'end': 10031}, 'R3': {'start': 9968, 'end': 10031}, 'R4': {'start': 9968, 'end': 10031}, 'R5': {'start': 10031, 'end': 10095}, 'R6': {'start': 10031, 'end': 10095}, 'R7': {'start': 10031, 'end': 10095}, 'R8': {'start': 10031, 'end': 10141}, 'R9': {'start': 10031, 'end': 10141}, 'R10': {'start': 10031, 'end': 10141}, 'R11': {'start': 10141, 'end': 10206}, 'R12': {'start': 10141, 'end': 10206}, 'R13': {'start': 10141, 'end': 10206}, 'R14': {'start': 10141, 'end': 10206}, 'R15': {'start': 10141, 'end': 10206}, 'R16': {'start': 10141, 'end': 10235}, 'R17': {'start': 10141, 'end': 10235}, 'R18': {'start': 10235, 'end': 10300}, 'R19': {'start': 10235, 'end': 10300}, 'R20': {'start': 10235, 'end': 10300}, 'R21': {'start': 10235, 'end': 10300}, 'R22': {'start': 10235, 'end': 10300}, 'R23': {'start': 10235, 'end': 10314}, 'R24': {'start': 10314, 'end': 10375}, 'R25': {'start': 10314, 'end': 10375}, 'R26': {'start': 10314, 'end':

AssertionError: 